In [1]:
from os.path import exists,basename
from json import load,dumps

In [2]:
from pytorch_transformers import BertTokenizer
from pytorch_transformers.modeling_bert import BertConfig, BertForSequenceClassification

In [3]:
import torch
import torch.nn as nn
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pathlib import Path
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'Tokenizer' from 'fastai.text' (/home/roger/.local/share/virtualenvs/welfare_state_analytics-0GZ2Cxn6/lib/python3.8/site-packages/fastai/text/__init__.py)

In [13]:

from fastai.text.transform import Tokenizer, BaseTokenizer, List, Vocab
from fastai.text.data import TextDataBunch
from fastai1.text import *
from fastai1.metrics import fbeta, accuracy

ImportError: cannot import name 'TextDataBunch' from 'fastai.text.data' (/home/roger/.local/share/virtualenvs/welfare_state_analytics-0GZ2Cxn6/lib/python3.8/site-packages/fastai/text/data.py)

In [5]:
from sklearn.metrics.classification import confusion_matrix
from sklearn.metrics import f1_score,fbeta_score

In [6]:
from confusion import get_confusion_plot, plot_confusion_matrix
from sklearn.metrics.classification import confusion_matrix
from random import random
from utils import Config, FastAiBertTokenizer, fill_categories, fill, accuracy, loss_batch_bert

ModuleNotFoundError: No module named 'fastai.text.transform'

In [ ]:
pd.set_option('display.max_columns', 5)

In [ ]:
data_file = '../data_swepub/E2.csv'

In [ ]:
N_PER_CATEGORY=None
N_MAX=None
seed = 42 # use constant seed for predictability
seed = seed or int(10000 * random()) 

In [ ]:
# load data
data = read_csv(data_file)

# randomize
data = data.sample(frac=1, random_state=seed)

# create 'text' column and put it first
data['text'] = data['title'] + " " + data['abstract'] 
data = data[data.columns.tolist()[:1] + data.columns.tolist()[-1:] + data.columns.tolist()[1:-1]]
data = data.fillna('')

text_col = 'text'
first_label_index=8 # <- be sure to use correct index
label_cols = list(data.columns[first_label_index:]) 

# filter data

# calculate sums
sums = DataFrame({ label: [ sum(data[label]) ] for label in label_cols })

train, val = train_test_split(data, train_size=0.6)
val, test = train_test_split(val, train_size=0.5)

# equalize categories?
if N_PER_CATEGORY:
    train = fill_categories(train, label_cols, n=N_PER_CATEGORY, random_state=seed+1)

if N_MAX:
    train = train.sample(n=N_MAX, random_state=seed+2)


In [ ]:
sums

In [ ]:
orgs = set()
for org in data['organisation']:
    for o in org.split(';'):
        if o.endswith('.se'):
            orgs.add(o)

In [ ]:
len(data), len(train), len(val), len(test)

In [ ]:
config = Config(
    testing=False,
    model_name="bert-large-cased",
    max_lr=3e-5,
    epochs=2,
    use_fp16=True,
    bs=64,
    discriminative=False,
    max_seq_len=256
)

In [ ]:
bert_tok = BertTokenizer.from_pretrained(config.model_name)

In [ ]:
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))

In [ ]:
fastai_tokenizer = Tokenizer(
                        tok_func=FastAiBertTokenizer(
                                    bert_tok, 
                                    max_seq_len=config.max_seq_len),
                        pre_rules=[],
                        post_rules=[])

In [ ]:
# read the data into a databunch 
databunch = TextDataBunch.from_df(".", train, val, test,
    tokenizer=fastai_tokenizer, vocab=fastai_bert_vocab,
    include_bos=False,
    include_eos=False, 
    text_cols=text_col, 
    label_cols=label_cols, 
    bs=config.bs,
    collate_fn=partial(pad_collate, pad_first=False))

In [ ]:
#from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained(config.model_name, num_labels=len(label_cols))

In [ ]:
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
# To change the loss_batch function in the loaded fastai module
module_basic_train = sys.modules['fastai.basic_train']
module_basic_train.loss_batch = loss_batch_bert
sys.modules['fastai.basic_train'] = module_basic_train

In [ ]:
f1 = partial(fbeta, beta=1)

In [ ]:
learner = Learner(
            databunch, bert_model,
            loss_func=loss_func,
            metrics=[ fbeta, f1, accuracy ]
            )

In [ ]:
if config.use_fp16:
    learner = learner.to_fp16()

In [ ]:
learner.callbacks.append(ShowGraph(learner))

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(config.epochs, max_lr=config.max_lr, moms=(0.8,0.7))

In [ ]:
preds, y_true = learner.get_preds()

In [ ]:
preds

In [ ]:
y_true

In [ ]:
dataset = basename(data_file)
dataset = dataset[0:dataset.index('.')]
accuracy = (y_true.argmax(axis=1) == preds.argmax(axis=1)).float().mean()
f1 = f1_score(y_true.argmax(axis=1), preds.argmax(axis=1), average='weighted')
f2 = fbeta_score(y_true.argmax(axis=1), preds.argmax(axis=1), beta=2, average='weighted')
cm = confusion_matrix(y_true.argmax(axis=1), preds.argmax(axis=1))

In [ ]:
title=f'Dataset={dataset}, model={config.model_name}, max_lr={config.max_lr}, {("seed=" + str(seed) + ", ") if seed else ""}{("N per category=" + str(N_PER_CATEGORY) + ", ") if N_PER_CATEGORY else ""}ntrain={len(train)}, nval={len(val)}, ntest={len(test)}, Acc={accuracy:.3f}, F1={f1:.3f}, F2={f2:.3f}'
c = get_confusion_plot(cm=cm, normalize=False, labels=list(label_cols), title=title)

In [ ]:
with open(f'results/result_{int(10000*random())}.txt', mode='w') as f:
    f.write(dumps(
        {
          'seed': seed,
          'dataset': basename(data_file),
          'train_size': len(train),
          'val_size': len(val),
          'test_size': len(val),
          'model': config.model_name,
          'max_lr': config.max_lr,
          'batch_size': config.bs,
          'max_seq_length': config.max_seq_len,
          'n_per_category': N_PER_CATEGORY,
          'n_max': N_MAX,
          'accuracy': float(accuracy),
          'f1': float(f1),
          'f2': float(f2),
          'labels': label_cols,
          'confusion': cm.tolist()
        }
    ))

In [ ]:
DataFrame(preds[0:10].tolist(), columns=label_cols).style.background_gradient(axis=1, cmap='Blues')

In [ ]:
DataFrame(y_true[0:10].tolist(), columns=label_cols).style.background_gradient(axis=1, cmap='Blues')